In [1]:
fo=open("chatbot.txt","r")
chatbot_data=fo.read()
fo.close()

In [2]:
list_of_sentences=chatbot_data.replace("\t"," ").split("\n")

In [3]:
import re
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
def ngram_to_list(sentences):
    corpus=[]
    count_is=0
    for i in sentences:
        review=re.sub("[^a-zA-z]"," ",i)
        corpus.append(review)
        if len(i)>count_is:
            count_is=len(i)
    cv=CountVectorizer(binary=True,ngram_range=(1,count_is))
    X=cv.fit_transform(corpus)
    list_ngrams=list(cv.vocabulary_)
    return list_ngrams


In [5]:
corpus_of_normal_text=ngram_to_list(list_of_sentences)
print(len(corpus_of_normal_text))

90834


In [6]:
import pandas as pd

# Creating dataframe of normal text

In [7]:
df_chat={}
df_chat["Response"]=corpus_of_normal_text
df_chat["Label"]=[0 for i in range(len(corpus_of_normal_text))]
df_chat=pd.DataFrame(df_chat)

In [8]:
print(df_chat.shape)
print("total no of unique values are :")
print(df_chat.nunique())
df_chat.head()

(90834, 2)
total no of unique values are :
Response    90834
Label           1
dtype: int64


,Response,Label
0,what,0
1,are,0
2,your,0
3,interests,0
4,am,0


# Creating dataframe of gibberish text

In [9]:
df_gibberish=pd.read_csv("Gibberish.csv")

In [10]:
print(df_gibberish.shape)
print("total no of unique values are :")
print(df_gibberish.nunique())
df_gibberish.head()

(3767, 2)
total no of unique values are :
Response    3516
Label          1
dtype: int64


,Response,Label
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1


In [11]:
list_of_gibberish=list(df_gibberish["Response"])

In [12]:
corpus_of_gibberish_text=ngram_to_list(list_of_gibberish)
print(len(corpus_of_gibberish_text))

23239


In [13]:
df_gibberish={}
df_gibberish["Response"]=corpus_of_gibberish_text
df_gibberish["Label"]=[1 for i in range(len(corpus_of_gibberish_text))]
df_gibberish=pd.DataFrame(df_gibberish)

In [14]:
print(df_gibberish.shape)
print("total no of unique values are :")
print(df_gibberish.nunique())
df_gibberish.head()

(23239, 2)
total no of unique values are :
Response    23239
Label           1
dtype: int64


,Response,Label
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1


# combining df_chat and df_gibberish

In [15]:
df_main=pd.concat([df_gibberish,df_chat],ignore_index=True)

In [16]:
print(df_main.shape)
print("total no of unique values are :")
print(df_main.nunique())
df_main.head()

(114073, 2)
total no of unique values are :
Response    113989
Label            2
dtype: int64


,Response,Label
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1


In [17]:
df_main.tail()

,Response,Label
114068,use sense similar to sonar to determine the lo...,0
114069,sense similar to sonar to determine the locati...,0
114070,dolphins use sense similar to sonar to determi...,0
114071,use sense similar to sonar to determine the lo...,0
114072,dolphins use sense similar to sonar to determi...,0


# converting of text to tokens

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [19]:
vocab=100000

In [20]:
sentences=list(df_chat["Response"])
len(sentences)

90834

In [21]:
fo=open("paragraph.txt","r")
fo_text=fo.read()
fo.close()

In [22]:
sentences+=fo_text.split()

In [23]:
len(sentences)

99853

In [24]:
tokenizer=Tokenizer(num_words=vocab,oov_token="<OOV>")

In [25]:
tokenizer.fit_on_texts(sentences)

In [26]:
len(tokenizer.word_index)

3643

In [27]:
sequences=tokenizer.texts_to_sequences(df_main["Response"])

In [61]:
max_len=0
for i in sequences:
    if len(i)>max_len:
        max_len=len(i)
print(max_len)

69


In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [62]:
padded=pad_sequences(sequences,maxlen=max_len)

In [63]:
padded

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       ...,
       [   0,    0,    0, ...,    3, 1401, 1514],
       [   0,    0,    0, ..., 1401, 1514, 1704],
       [   0,    0,    0, ..., 1401, 1514, 1704]])

# splitting training and test data

In [32]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(padded, df_main["Label"], test_size=0.30,random_state=0)

In [40]:
print("the shape of X_train is : ",X_train.shape)
print("the shape of X_test is : ",X_test.shape)
print("the shape of y_train is : ",y_train.shape)
print("the shape of y_test is : ",y_test.shape)

the shape of X_train is :  (79851, 69)
the shape of X_test is :  (34222, 69)
the shape of y_train is :  (79851,)
the shape of y_test is :  (34222,)


In [41]:
print(y_test.value_counts())

0    27284
1     6938
Name: Label, dtype: int64


In [42]:
import keras

In [43]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab, 64),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(1, activation="sigmoid")
])

In [44]:
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=["acc"])

In [45]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test),  verbose=1)

2496/2496 [==============================] - 360s 143ms/step - loss: 0.0466 - acc: 0.9840 - val_loss: 0.0065 - val_acc: 0.9986


In [46]:
y_predict=model.predict(X_test)

1070/1070 [==============================] - 8s 6ms/step


In [47]:
y_predict=y_predict>0.5

In [48]:
from sklearn.metrics import accuracy_score,classification_report

In [49]:
accuracy_score(y_predict,y_test)

0.9986266144585354

In [50]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00     27331
        True       0.99      1.00      1.00      6891

    accuracy                           1.00     34222
   macro avg       1.00      1.00      1.00     34222
weighted avg       1.00      1.00      1.00     34222



# testing the text

In [76]:
def testing_text(text):
    dummy=[]
    dummy.append(text)
    sequences=tokenizer.texts_to_sequences(dummy)
    padded=pad_sequences(sequences,maxlen=max_len)
    y=model.predict(padded)
    y=y>0.5
    if y:
        print("text is gibberish")
    else:
        print("text is normal")

In [85]:
testing_text("hello morning")

1/1 [==============================] - 0s 48ms/step
text is normal
